In [229]:
from transformers import DistilBertTokenizerFast, DistilBertTokenizer, DistilBertForQuestionAnswering
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

In [230]:
def askDistilBertOne(context, question):
    # need to change this so that accomadates text larger than 512 words
    model_path = 'model/distilbert-custom'
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
    encoding = tokenizer([context], [question], padding='max_length', max_length=512)
    
    model = DistilBertForQuestionAnswering.from_pretrained(model_path)
    train_loader = DataLoader(dataset, batch_size=1)
    
    model.to(torch.device('cpu'))
    model.eval()
    
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        with torch.no_grad():

            input_ids = batch['input_ids'].to(torch.device('cpu'))
            attention_mask = batch['attention_mask'].to(torch.device('cpu'))
            outputs = model(input_ids, attention_mask=attention_mask)
        
            answer_start_index = torch.argmax(outputs['start_logits'], dim=1)
            answer_end_index = torch.argmax(outputs['end_logits'], dim=1)
    
    # some of the tokens include punctuation char so need to make sure there are no spaces around them
    output = ""
    for i in range(answer_start_index, answer_end_index + 1):
        output += tokenizer.decode(encoding['input_ids'][0][i], skip_special_tokens=True).replace(" ", "") + " "
        output = output[:-1]
    return output

In [231]:
context = "To better assess the impact of the current wave — which projections suggest could claim more than 1 million lives in 2023 — The Washington Post tracked hundreds of posts on popular Chinese platforms, including Weibo and Douyin, and reviewed material that was reposted on Twitter and other sites. The Post’s preliminary analysis found evidence of overwhelmed health-care facilities in major cities, particularly along the country’s heavily populated east coast."
question = "Where was the reviewed material reposted?"
output = askDistilBertOne(context, question)
output # wrong answer need to figure out what changed

100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.18it/s]


'posttrackedhundredsofposts'

In [232]:
def askDistilBertTwo(context, question):
    model_path = 'model/distilbert-custom'
    tokenizer = DistilBertTokenizer.from_pretrained(model_path)
    model = DistilBertForQuestionAnswering.from_pretrained(model_path)
    inputs = tokenizer(context, question, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()
    
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    return tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

In [233]:
context = "To better assess the impact of the current wave — which projections suggest could claim more than 1 million lives in 2023 — The Washington Post tracked hundreds of posts on popular Chinese platforms, including Weibo and Douyin, and reviewed material that was reposted on Twitter and other sites. The Post’s preliminary analysis found evidence of overwhelmed health-care facilities in major cities, particularly along the country’s heavily populated east coast."
question = "Where was the reviewed material reposted?"
output = askDistilBertTwo(context, question)
output

'twitter and other sites.'